In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/ST/stargan

Mounted at /content/drive
/content/drive/MyDrive/ST/stargan


In [2]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import LambdaLR
import torch.optim as optim
import os
import csv

seed = 2710
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

In [6]:
class TSTRClassifier(nn.Module):
    def __init__(self, num_timesteps=128, num_channels=3, num_classes=5):
        super(TSTRClassifier, self).__init__()

        self.conv1 = nn.Conv1d(num_channels, 16, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(64)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn4 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.25)

        self.fc_shared = nn.Linear(num_timesteps * 8, 100)

        self.fc_class = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.relu(self.fc_shared(x))

        # Final output for class prediction
        class_outputs = self.fc_class(x)
        return class_outputs


def get_df_data(dataset_name, class_idx, num_train_domains):

    # Load the dataset
    with open(f'data/{dataset_name}.pkl', 'rb') as f:
        x, y, k = pickle.load(f)

    with open(f'data/{dataset_name}_fs.pkl', 'rb') as f:
        fs = pickle.load(f)

    # Filter out the samples that are used for finetuning
    x = x[fs == 0]
    y = y[fs == 0]
    k = k[fs == 0]

    x_ = x[(y == class_idx) & (k < num_train_domains)]
    y_ = y[(y == class_idx) & (k < num_train_domains)]
    k_ = k[(y == class_idx) & (k < num_train_domains)]

    return x_, y_, k_


def get_dp_data(dataset_name, class_idx, num_train_domains):

    # Load the dataset
    with open(f'data/{dataset_name}.pkl', 'rb') as f:
        x, y, k = pickle.load(f)

    with open(f'data/{dataset_name}_fs.pkl', 'rb') as f:
        fs = pickle.load(f)

    # Filter out the samples that are used for finetuning
    x = x[fs == 0]
    y = y[fs == 0]
    k = k[fs == 0]

    x_ = x[(y == class_idx) & (k >= num_train_domains)]
    y_ = y[(y == class_idx) & (k >= num_train_domains)]
    k_ = k[(y == class_idx) & (k >= num_train_domains)]

    return x_, y_, k_


def get_fs_data(dataset, src_class, domain):

    if dataset == 'realworld':
        dataset_name = 'realworld_128_3ch_4cl'
        class_names = ['WAL', 'RUN', 'CLD', 'CLU']
    elif dataset == 'cwru':
        dataset_name = 'cwru_256_3ch_5cl'
        class_names = ['IR', 'Ball', 'OR_centred', 'OR_orthogonal', 'OR_opposite']

    class_idx = class_names.index(src_class)

    # Load the dataset
    with open(f'data/{dataset_name}.pkl', 'rb') as f:
        x, y, k = pickle.load(f)

    with open(f'data/{dataset_name}_fs.pkl', 'rb') as f:
        fs = pickle.load(f)

    x = x[fs == 1]
    y = y[fs == 1]
    k = k[fs == 1]

    x_ = x[(y != class_idx) & (k == domain)]
    y_ = y[(y != class_idx) & (k == domain)]
    k_ = k[(y != class_idx) & (k == domain)]

    return x_, y_, k_




def remap_labels(y):
    label_map = {clss: i for i, clss in enumerate(np.unique(y))}
    return np.array([label_map[clss] for clss in y])


def setup_training(x_tr, y_tr, x_val, y_val, batch_size=64):
    # Convert numpy arrays to torch tensors
    x_train_tensor = torch.tensor(x_tr, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_tr, dtype=torch.long)
    x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)

    # Create datasets and loaders
    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader


def train_model(model, train_loader, val_loader, loss_fn, optimizer, epochs=300):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    loss_train = []
    loss_val = []
    accuracy_val = []
    best_loss = np.inf
    best_accuracy = 0

    # Set up linear learning rate decay
    lambda_lr = lambda epoch: 1 - epoch / epochs
    scheduler = LambdaLR(optimizer, lr_lambda=lambda_lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        total_loss /= len(train_loader)
        loss_train.append(total_loss)

        # Update learning rate
        scheduler.step()

        val_accuracy, val_loss = evaluate_model(model, val_loader, loss_fn)
        if val_accuracy >= best_accuracy:
            best_epoch = epoch
            best_accuracy = val_accuracy
            best_loss = val_loss
            best_model_state = model.state_dict().copy()
        loss_val.append(val_loss)
        accuracy_val.append(val_accuracy)

        current_lr = scheduler.get_last_lr()[0]
        print(f"Epoch {epoch + 1}/{epochs} - Train loss: {total_loss:.4f} - Val loss: {val_loss:.4f} - Val accuracy: {val_accuracy:.4f} - LR: {current_lr:.8f}")

    print(f"Best epoch: {best_epoch + 1} - Best val accuracy: {best_accuracy:.4f} - Best val loss: {best_loss:.4f}")

    return best_model_state


def evaluate_model(model, test_loader, loss_fn):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            total_loss += loss.item()

            _, predicted_labels = torch.max(outputs, 1)
            correct_predictions += (predicted_labels == y_batch).sum().item()
            total_predictions += len(y_batch)

    total_loss /= len(test_loader)
    accuracy = correct_predictions / total_predictions

    return accuracy, total_loss


def calculate_tstr_score(x_train, y_train, x_test, y_test, k_test, source, dataset):
    assert np.array_equal(np.unique(y_train), np.unique(y_test)), "Train and test labels do not match"
    # Remap labels
    y_train = remap_labels(y_train)
    y_test = remap_labels(y_test)

    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2710, stratify=y_train, shuffle=True)
    tr_loader, val_loader = setup_training(x_tr, y_tr, x_val, y_val, batch_size=64)

    model = TSTRClassifier(num_timesteps=x_train.shape[2], num_channels=x_train.shape[1], num_classes=len(np.unique(y_train)))
    loss_fn = nn.CrossEntropyLoss()
    initial_lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    best_model_state = train_model(model, tr_loader, val_loader, loss_fn, optimizer, epochs=50)
    best_model = TSTRClassifier(num_timesteps=x_train.shape[2], num_channels=x_train.shape[1], num_classes=len(np.unique(y_train)))
    best_model.load_state_dict(best_model_state)
    acc, loss = evaluate_model_on_domains(best_model, x_test, y_test, k_test, loss_fn, source, dataset)

    return acc, loss


def evaluate_model_on_domains(best_model, x_test, y_test, k_test, loss_fn, source, dataset):
    accs = []
    losses = []
    for domain in np.unique(k_test):
        x_fs_domain, y_fs_domain, k_fs_domain = get_fs_data(dataset, source, domain)
        print(f"Source: {source} - Domain: {domain} - Num samples: {len(y_fs_domain)}")
        y_fs_domain = remap_labels(y_fs_domain)
        fine_tuned_model_state = fine_tune_model(best_model, x_fs_domain, y_fs_domain)

        fine_tuned_model = TSTRClassifier(num_timesteps=x_test.shape[2], num_channels=x_test.shape[1], num_classes=len(np.unique(y_test)))
        fine_tuned_model.load_state_dict(fine_tuned_model_state)

        x_domain = x_test[k_test == domain]
        y_domain = y_test[k_test == domain]

        x_domain_tensor = torch.tensor(x_domain, dtype=torch.float32)
        y_domain_tensor = torch.tensor(y_domain, dtype=torch.long)
        domain_dataset = TensorDataset(x_domain_tensor, y_domain_tensor)
        domain_loader = DataLoader(domain_dataset, batch_size=64, shuffle=False)

        acc, loss = evaluate_model(fine_tuned_model, domain_loader, loss_fn)
        print(f'Source: {source} - Domain: {domain} - Accuracy: {acc:.2f} - Loss: {loss:.4f}')
        save_score(acc, loss, source, domain, 'LB_real_FT_fs', dataset)
        accs.append(acc)
        losses.append(loss)
    return np.mean(accs), np.mean(losses)


def fine_tune_model(best_model, x_fs_domain, y_fs_domain):
    x_tr, x_val, y_tr, y_val = train_test_split(x_fs_domain, y_fs_domain, test_size=0.2, random_state=2710, stratify=y_fs_domain, shuffle=True)
    tr_loader, val_loader = setup_training(x_tr, y_tr, x_val, y_val, batch_size=64)

    loss_fn = nn.CrossEntropyLoss()
    initial_lr = 1e-5
    optimizer = optim.Adam(best_model.parameters(), lr=initial_lr)

    fine_tuned_model_state = train_model(best_model, tr_loader, val_loader, loss_fn, optimizer, epochs=50)

    return fine_tuned_model_state


def save_score(accuracy, loss, source, domain, name, dataset):
    eval_dir = 'bounds_fs'
    # Ensure the directory exists
    os.makedirs(eval_dir, exist_ok=True)
    # Path to the CSV file
    file_path = os.path.join(eval_dir, f'{name}_{dataset}.csv')
    # Check if the file exists
    file_exists = os.path.exists(file_path)

    # Open the file in append mode if it exists, or write mode if it doesn't
    with open(file_path, mode='a' if file_exists else 'w', newline='') as file:
        writer = csv.writer(file)
        # If the file does not exist, write the header
        if not file_exists:
            writer.writerow(['source', 'domain', 'accuracy', 'loss'])
        # Write the data rows
        writer.writerow([source, domain, accuracy, loss])


In [4]:
dataset = 'realworld'

if dataset == 'realworld':
    dataset_name = 'realworld_128_3ch_4cl'
    num_df_domains = 10
    num_dp_domains = 5
    num_classes = 4
    class_names = ['WAL', 'RUN', 'CLD', 'CLU']

elif dataset == 'cwru':
    dataset_name = 'cwru_256_3ch_5cl'
    num_df_domains = 4
    num_dp_domains = 4
    num_classes = 5
    class_names = ['IR', 'Ball', 'OR_centred', 'OR_orthogonal', 'OR_opposite']

classes_dict = {clss: i for i, clss in enumerate(class_names)}

accs = []

for src_class in class_names:
    trg_classes = [clss for clss in class_names if clss != src_class]

    x_df = []
    y_df = []
    k_df = []

    x_dp = []
    y_dp = []
    k_dp = []

    for trg_class in trg_classes:
        x_df_, y_df_, k_df_ = get_df_data(dataset_name, classes_dict[trg_class], num_df_domains)
        x_dp_, y_dp_, k_dp_ = get_dp_data(dataset_name, classes_dict[trg_class], num_df_domains)

        x_df.append(x_df_)
        y_df.append(y_df_)
        k_df.append(k_df_)

        x_dp.append(x_dp_)
        y_dp.append(y_dp_)
        k_dp.append(k_dp_)

    x_df = np.concatenate(x_df, axis=0)
    y_df = np.concatenate(y_df, axis=0)
    k_df = np.concatenate(k_df, axis=0)

    x_dp = np.concatenate(x_dp, axis=0)
    y_dp = np.concatenate(y_dp, axis=0)
    k_dp = np.concatenate(k_dp, axis=0)

    acc, loss = calculate_tstr_score(x_df, y_df, x_dp, y_dp, k_dp, src_class, dataset)
    accs.append(acc)

    print(f'{src_class} -> {trg_classes}: {acc:.2f}\n\n')

print(f'Mean accuracy: {np.mean(accs):.2f}')

Epoch 1/50 - Train loss: 0.5190 - Val loss: 0.2824 - Val accuracy: 0.9206 - LR: 0.00009800
Epoch 2/50 - Train loss: 0.1973 - Val loss: 0.1389 - Val accuracy: 0.9594 - LR: 0.00009600
Epoch 3/50 - Train loss: 0.1419 - Val loss: 0.1099 - Val accuracy: 0.9650 - LR: 0.00009400
Epoch 4/50 - Train loss: 0.1103 - Val loss: 0.0981 - Val accuracy: 0.9707 - LR: 0.00009200
Epoch 5/50 - Train loss: 0.0951 - Val loss: 0.0904 - Val accuracy: 0.9679 - LR: 0.00009000
Epoch 6/50 - Train loss: 0.0847 - Val loss: 0.0772 - Val accuracy: 0.9754 - LR: 0.00008800
Epoch 7/50 - Train loss: 0.0745 - Val loss: 0.0714 - Val accuracy: 0.9754 - LR: 0.00008600
Epoch 8/50 - Train loss: 0.0612 - Val loss: 0.0628 - Val accuracy: 0.9735 - LR: 0.00008400
Epoch 9/50 - Train loss: 0.0543 - Val loss: 0.0603 - Val accuracy: 0.9764 - LR: 0.00008200
Epoch 10/50 - Train loss: 0.0511 - Val loss: 0.0590 - Val accuracy: 0.9754 - LR: 0.00008000
Epoch 11/50 - Train loss: 0.0487 - Val loss: 0.0510 - Val accuracy: 0.9783 - LR: 0.000078

In [7]:
dataset = 'cwru'

if dataset == 'realworld':
    dataset_name = 'realworld_128_3ch_4cl'
    num_df_domains = 10
    num_dp_domains = 5
    num_classes = 4
    class_names = ['WAL', 'RUN', 'CLD', 'CLU']

elif dataset == 'cwru':
    dataset_name = 'cwru_256_3ch_5cl'
    num_df_domains = 4
    num_dp_domains = 4
    num_classes = 5
    class_names = ['IR', 'Ball', 'OR_centred', 'OR_orthogonal', 'OR_opposite']

classes_dict = {clss: i for i, clss in enumerate(class_names)}

accs = []

for src_class in class_names:
    trg_classes = [clss for clss in class_names if clss != src_class]

    x_df = []
    y_df = []
    k_df = []

    x_dp = []
    y_dp = []
    k_dp = []

    for trg_class in trg_classes:
        x_df_, y_df_, k_df_ = get_df_data(dataset_name, classes_dict[trg_class], num_df_domains)
        x_dp_, y_dp_, k_dp_ = get_dp_data(dataset_name, classes_dict[trg_class], num_df_domains)

        x_df.append(x_df_)
        y_df.append(y_df_)
        k_df.append(k_df_)

        x_dp.append(x_dp_)
        y_dp.append(y_dp_)
        k_dp.append(k_dp_)

    x_df = np.concatenate(x_df, axis=0)
    y_df = np.concatenate(y_df, axis=0)
    k_df = np.concatenate(k_df, axis=0)

    x_dp = np.concatenate(x_dp, axis=0)
    y_dp = np.concatenate(y_dp, axis=0)
    k_dp = np.concatenate(k_dp, axis=0)

    acc, loss = calculate_tstr_score(x_df, y_df, x_dp, y_dp, k_dp, src_class, dataset)
    accs.append(acc)

    print(f'{src_class} -> {trg_classes}: {acc:.2f}\n\n')

print(f'Mean accuracy: {np.mean(accs):.2f}')

Epoch 1/50 - Train loss: 0.4299 - Val loss: 0.3305 - Val accuracy: 0.8355 - LR: 0.00009800
Epoch 2/50 - Train loss: 0.0717 - Val loss: 0.1225 - Val accuracy: 0.9523 - LR: 0.00009600
Epoch 3/50 - Train loss: 0.0229 - Val loss: 0.2532 - Val accuracy: 0.8853 - LR: 0.00009400
Epoch 4/50 - Train loss: 0.0106 - Val loss: 0.5729 - Val accuracy: 0.7924 - LR: 0.00009200
Epoch 5/50 - Train loss: 0.0060 - Val loss: 0.0110 - Val accuracy: 0.9973 - LR: 0.00009000
Epoch 6/50 - Train loss: 0.0042 - Val loss: 0.0033 - Val accuracy: 1.0000 - LR: 0.00008800
Epoch 7/50 - Train loss: 0.0046 - Val loss: 0.0421 - Val accuracy: 0.9828 - LR: 0.00008600
Epoch 8/50 - Train loss: 0.0028 - Val loss: 0.0013 - Val accuracy: 1.0000 - LR: 0.00008400
Epoch 9/50 - Train loss: 0.0039 - Val loss: 2.3995 - Val accuracy: 0.7507 - LR: 0.00008200
Epoch 10/50 - Train loss: 0.0023 - Val loss: 0.0114 - Val accuracy: 0.9940 - LR: 0.00008000
Epoch 11/50 - Train loss: 0.0022 - Val loss: 0.0010 - Val accuracy: 1.0000 - LR: 0.000078